# k-Abelian Complexity of Pell sequence from 1 to 5

## Initial Numeration System Setup

First we define the subsitution, its numeration system and its fixpoint
sequence.

In [2]:
%%python
from licofage.kit import *
import os
setparams(True, True, os.environ["WALNUT_HOME"])

s = subst('001/0')
ns = address(s, "pell")
ns.gen_ns()
ns.gen_word_automaton()

>>> Address automaton for subst(0->001, 1->0)
  DT polynomial: X^2-2X-1 (n=2)
  θ=2.414213562373095
  DT automaton: 2 states, 2 final states, 4 transitions.

>>> address(subst(0->001, 1->0), 'pell') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 4 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.N9a7399yLZ/Walnut/Custom Bases/msd_pell.txt in format Walnut
  +address(subst(0->001, 1->0), 'pell')+address(subst(0->001, 1->0), 'pell')-address(subst(0->001, 1->0), 'pell') is generating it's dfa under name msd_pell_addition
  Combination polynomial is X^2-2X-1
  θ=2.414213562373095
  Combination automaton: 8 states, 8 final states, 64 transitions.
  (P,u) vector bound: (0.0, 0.0) + (1.8106601717798219, 3.5177669529663698)*C
  Fast poly bound: C in [(-2, -2), (4, 4)] => (14.485281374238575, 28.142135623730958)
Expanding DFA...done.
  Flattened automaton : 276 states, 11 final states, 1080 transitions.
  Trimmed flattened automaton : 2

Then we setup a factor comparison predicate in Walnut and a first factor
occurence predicate.

In [3]:
def cut "?msd_pell i<=u & j<=v & u+j=v+i & u<n+i & v<n+j":
def feq_pell "?msd_pell ~(Eu,v $cut(i,j,n,u,v) & Pell[u]!=Pell[v])":
eval comp_pell n "?msd_pell Aj $feq_pell(i,j,n) => i<=j":

i<=u:6 states - 6ms
 j<=v:6 states - 1ms
  (i<=u&j<=v):36 states - 3ms
   (u+j)=(v+i):70 states - 22ms
    ((i<=u&j<=v)&(u+j)=(v+i)):74 states - 10ms
     u<(n+i):29 states - 4ms
      (((i<=u&j<=v)&(u+j)=(v+i))&u<(n+i)):329 states - 40ms
       v<(n+j):29 states - 3ms
        ((((i<=u&j<=v)&(u+j)=(v+i))&u<(n+i))&v<(n+j)):329 states - 17ms
Total computation time: 113ms.

Pell[u]!=Pell[v]:4 states - 0ms
 (cut(i,j,n,u,v))&Pell[u]!=Pell[v]):329 states - 13ms
  (E u , v (cut(i,j,n,u,v))&Pell[u]!=Pell[v])):23 states - 41ms
   ~(E u , v (cut(i,j,n,u,v))&Pell[u]!=Pell[v])):11 states - 2ms
Total computation time: 72ms.

i<=j:6 states - 0ms
 (feq_pell(i,j,n))=>i<=j):25 states - 3ms
  (A j (feq_pell(i,j,n))=>i<=j)):10 states - 2ms
Total computation time: 6ms.


From there we can define the boundary condition `bordercond`, as
explained in Fici-Puzynina-2023 section 8.1.

In [4]:
def bordercond "?msd_pell (k<=n => $feq_pell(i,j,k-1)) & (n<k => $feq_pell(i,j,n))":

k<=n:6 states - 0ms
computed ~:1 states - 6ms
computed ~:2 states - 0ms
 (k<=n=>feq_pell(i,j,(k-1)))):61 states - 9ms
  n<k:6 states - 0ms
   (n<k=>feq_pell(i,j,n))):61 states - 8ms
    ((k<=n=>feq_pell(i,j,(k-1))))&(n<k=>feq_pell(i,j,n)))):44 states - 3ms
Total computation time: 29ms.


## 1-abelian complexity

Define the 1-block map of pell and construct the conversion predicate
between both numeration systems.

In [5]:
%%python
s1 = s
ns1 = ns

### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Pell.

In [6]:
%%python
for (i,a) in enumerate(ns1.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s1, ns1.ns, **w)
    (parikh - ns1).gen_dfa(f"pellb1p{i}")

>>> Address automaton for subst(0->001, 1->0)
  DT polynomial: X^2-2X-1 (n=2)
  θ=2.414213562373095
  DT automaton: 2 states, 2 final states, 4 transitions.

>>> +address(subst(0->001, 1->0), 'pell', '_': 0, '0': 1)-address(subst(0->001, 1->0), 'pell') is generating it's dfa under name pellb1p0
  Combination polynomial is X^2-2X-1
  θ=2.414213562373095
  Combination automaton: 4 states, 4 final states, 16 transitions.
  (P,u) vector bound: (0.0, 0.0) + (1.8106601717798219, 3.5177669529663698)*C
  Fast poly bound: C in [(-2, -2), (0, 2)] => (7.242640687119287, 14.071067811865479)
Expanding DFA...done.
  Flattened automaton : 78 states, 5 final states, 148 transitions.
  Trimmed flattened automaton : 9 states, 5 final states, 19 transitions.
  Real bound is [1, 2] (vs [7.242640687119287, 14.071067811865479]).
  After minimization: 8 states, 5 final states, 17 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.N9a7399yLZ/Walnut/Automata Library/pellb1p0.txt in for

### Compute the complexity function

Use Walnut to generate a linear representation for the 1-abelian
complexity.

In [7]:
def fac0 "?msd_pell Ex,y $pellb1p0(i,x) & $pellb1p0(i+n,y) & z+x=y":
def fac1 "?msd_pell Ex,y $pellb1p1(i,x) & $pellb1p1(i+n,y) & z+x=y":

(pellb1p0(i,x))&pellb1p0((i+n),y))):114 states - 4ms
 (z+x)=y:16 states - 1ms
  ((pellb1p0(i,x))&pellb1p0((i+n),y)))&(z+x)=y):154 states - 7ms
   (E x , y ((pellb1p0(i,x))&pellb1p0((i+n),y)))&(z+x)=y)):35 states - 1ms
Total computation time: 20ms.

(pellb1p1(i,x))&pellb1p1((i+n),y))):143 states - 1ms
 (z+x)=y:16 states - 1ms
  ((pellb1p1(i,x))&pellb1p1((i+n),y)))&(z+x)=y):253 states - 9ms
   (E x , y ((pellb1p1(i,x))&pellb1p1((i+n),y)))&(z+x)=y)):44 states - 2ms
Total computation time: 15ms.


In [8]:
def min0 "?msd_pell Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_pell Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):132 states - 35ms
  (A j , y (fac0(j,n,y))=>y>=x)):13 states - 26ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):29 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):11 states - 0ms
Total computation time: 62ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):174 states - 40ms
  (A j , y (fac1(j,n,y))=>y>=x)):14 states - 33ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):34 states - 1ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 76ms.


In [9]:
def diff0 "?msd_pell Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_pell Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):55 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):53 states - 2ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):14 states - 0ms
Total computation time: 4ms.

(min1(n,x))&fac1(i,n,y))):58 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):55 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):14 states - 0ms
Total computation time: 2ms.


In [10]:
def abeq_pell1 "?msd_pell (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z))":

(diff0(i,n,z))&diff0(j,n,z))):46 states - 1ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):37 states - 1ms
  (diff1(i,n,z))&diff1(j,n,z))):46 states - 0ms
   (E z (diff1(i,n,z))&diff1(j,n,z)))):37 states - 1ms
    ((E z (diff0(i,n,z))&diff0(j,n,z))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):37 states - 1ms
Total computation time: 5ms.


In [11]:
eval comp_pell1 n "?msd_pell Aj $abeq_pell1(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_pell1(i,j,n))=>i<=j):54 states - 5ms
  (A j (abeq_pell1(i,j,n))=>i<=j)):4 states - 3ms
Total computation time: 8ms.


### Apply the semigroup trick

In [12]:
%SGT comp_pell1 msd_pell Comp_pell1

Reading Result/comp_pell1.mpl
Rank: 3
4 states
values: [0, 1, 2]
Writing Word Automata Library/Comp_pell1.txt
Done.

## 2-abelian complexity

Define the 2-block map of pell and construct the conversion predicate
between both numeration systems.

In [13]:
%%python
s2 = block(s, 2)
ns2 = address(s2, "pellb2")
ns2.gen_ns()
(ns-ns2).gen_dfa("conv_pell_pellb2")

>>> Address automaton for block(subst(0->001, 1->0), 2)
  DT polynomial: X^2-2X-1 (n=2)
  θ=2.414213562373095
  DT automaton: 3 states, 3 final states, 7 transitions.

>>> address(block(subst(0->001, 1->0), 2), 'pellb2') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 4 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.N9a7399yLZ/Walnut/Custom Bases/msd_pellb2.txt in format Walnut
  +address(block(subst(0->001, 1->0), 2), 'pellb2')+address(block(subst(0->001, 1->0), 2), 'pellb2')-address(block(subst(0->001, 1->0), 2), 'pellb2') is generating it's dfa under name msd_pellb2_addition
  Combination polynomial is X^2-2X-1
  θ=2.414213562373095
  Combination automaton: 27 states, 27 final states, 343 transitions.
  (P,u) vector bound: (0.0, 0.0) + (1.8106601717798219, 3.5177669529663698)*C
  Fast poly bound: C in [(-2, -2), (4, 4)] => (14.485281374238575, 28.142135623730958)
Expanding DFA...done.
  Flattened automaton : 616 states, 24 

Translate the border condition predicate into the current numeration
system.

In [14]:
def bordercond2 "?msd_pellb2 (?msd_pell Eii,jj,kk,nn 
($conv_pell_pellb2(?msd_pell ii, ?msd_pellb2 i) & 
$conv_pell_pellb2(?msd_pell jj, ?msd_pellb2 j) & 
$conv_pell_pellb2(?msd_pell kk, ?msd_pellb2 k) & 
$conv_pell_pellb2(?msd_pell nn, ?msd_pellb2 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_pell_pellb2(ii,i))&conv_pell_pellb2(jj,j))):4 states - 1ms
 ((conv_pell_pellb2(ii,i))&conv_pell_pellb2(jj,j)))&conv_pell_pellb2(kk,k))):8 states - 0ms
  (((conv_pell_pellb2(ii,i))&conv_pell_pellb2(jj,j)))&conv_pell_pellb2(kk,k)))&conv_pell_pellb2(nn,n))):16 states - 6ms
   ((((conv_pell_pellb2(ii,i))&conv_pell_pellb2(jj,j)))&conv_pell_pellb2(kk,k)))&conv_pell_pellb2(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 87ms
    (E ii , jj , kk , nn ((((conv_pell_pellb2(ii,i))&conv_pell_pellb2(jj,j)))&conv_pell_pellb2(kk,k)))&conv_pell_pellb2(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 4ms
Total computation time: 99ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Pellb2.

In [15]:
%%python
for (i,a) in enumerate(ns2.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s2, ns2.ns, **w)
    (parikh - ns2).gen_dfa(f"pellb2p{i}")

>>> Address automaton for block(subst(0->001, 1->0), 2)
  DT polynomial: X^3-2X^2-X (n=3)
  θ=2.414213562373095 (X^1)
  DT automaton: 3 states, 3 final states, 7 transitions.

>>> +address(block(subst(0->001, 1->0), 2), 'pellb2', '_': 0, 'a': 1)-address(block(subst(0->001, 1->0), 2), 'pellb2') is generating it's dfa under name pellb2p0
  Combination polynomial is X^3-2X^2-X
  θ=2.414213562373095 (X^1)
  Combination automaton: 9 states, 9 final states, 49 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.164213562373096, 8.992640687119287, 18.235281374238575)*C
  Fast poly bound: C in [(0, -3, -2), (1, 0, 1)] => (25.82106781186548, 44.96320343559644, 91.17640687119288)
Expanding DFA...done.
  Flattened automaton : 311 states, 9 final states, 696 transitions.
  Trimmed flattened automaton : 14 states, 9 final states, 30 transitions.
  Real bound is [1, 2, 6] (vs [25.82106781186548, 44.96320343559644, 91.17640687119288]).
  After minimization: 7 states, 4 final states, 17 transition

### Compute the complexity function

Use Walnut to generate a linear representation for the 2-abelian
complexity.

In [16]:
def fac0 "?msd_pellb2 Ex,y $pellb2p0(i,x) & $pellb2p0(i+n,y) & z+x=y":
def fac1 "?msd_pellb2 Ex,y $pellb2p1(i,x) & $pellb2p1(i+n,y) & z+x=y":
def fac2 "?msd_pellb2 Ex,y $pellb2p2(i,x) & $pellb2p2(i+n,y) & z+x=y":

(pellb2p0(i,x))&pellb2p0((i+n),y))):68 states - 0ms
 (z+x)=y:16 states - 0ms
  ((pellb2p0(i,x))&pellb2p0((i+n),y)))&(z+x)=y):122 states - 3ms
   (E x , y ((pellb2p0(i,x))&pellb2p0((i+n),y)))&(z+x)=y)):32 states - 1ms
Total computation time: 6ms.

(pellb2p1(i,x))&pellb2p1((i+n),y))):191 states - 2ms
 (z+x)=y:16 states - 0ms
  ((pellb2p1(i,x))&pellb2p1((i+n),y)))&(z+x)=y):331 states - 9ms
   (E x , y ((pellb2p1(i,x))&pellb2p1((i+n),y)))&(z+x)=y)):52 states - 2ms
Total computation time: 15ms.

(pellb2p2(i,x))&pellb2p2((i+n),y))):143 states - 1ms
 (z+x)=y:16 states - 1ms
  ((pellb2p2(i,x))&pellb2p2((i+n),y)))&(z+x)=y):253 states - 6ms
   (E x , y ((pellb2p2(i,x))&pellb2p2((i+n),y)))&(z+x)=y)):44 states - 1ms
Total computation time: 10ms.


In [17]:
def min0 "?msd_pellb2 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_pellb2 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_pellb2 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":

y>=x:6 states - 1ms
 (fac0(j,n,y))=>y>=x):137 states - 30ms
  (A j , y (fac0(j,n,y))=>y>=x)):10 states - 26ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):24 states - 1ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):9 states - 0ms
Total computation time: 58ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):191 states - 39ms
  (A j , y (fac1(j,n,y))=>y>=x)):14 states - 31ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):41 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 71ms.

y>=x:6 states - 1ms
 (fac2(j,n,y))=>y>=x):174 states - 33ms
  (A j , y (fac2(j,n,y))=>y>=x)):14 states - 26ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):34 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 60ms.


In [18]:
def diff0 "?msd_pellb2 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_pellb2 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_pellb2 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):48 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):46 states - 1ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):18 states - 0ms
Total computation time: 2ms.

(min1(n,x))&fac1(i,n,y))):69 states - 1ms
 (z+x)=y:16 states - 2ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):66 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):16 states - 0ms
Total computation time: 4ms.

(min2(n,x))&fac2(i,n,y))):58 states - 0ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):55 states - 2ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):14 states - 0ms
Total computation time: 2ms.


In [19]:
def abeq_pellb2 "?msd_pellb2 $bordercond2(i,j,2,n+1) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z))":
def abeq_pell2 "?msd_pell (n<1 & $feq_pell(i,j,n)) | (n>=1 & (?msd_pellb2 Eii,jj,nn ($conv_pell_pellb2(?msd_pell i, ?msd_pellb2 ii) & $conv_pell_pellb2(?msd_pell j, ?msd_pellb2 jj) & $conv_pell_pellb2(?msd_pell n, ?msd_pellb2 nn) & $abeq_pellb2(ii,jj,nn-1))))":

computed ~:1 states - 0ms
computed ~:2 states - 1ms
(diff0(i,n,z))&diff0(j,n,z))):64 states - 1ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):51 states - 1ms
  (bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):51 states - 1ms
   (diff1(i,n,z))&diff1(j,n,z))):60 states - 0ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):51 states - 1ms
     ((bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):51 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):46 states - 1ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):37 states - 0ms
        (((bordercond2(i,j,2,(n+1)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):51 states - 1ms
Total computation time: 9ms.

n<1:1 states - 1ms
 (n<1&feq_pell(i,j,n))):4 states - 0ms
  n>=1:3 states - 0ms
   (conv_pell_pellb2(i,ii))&conv_pell_pellb2(j,jj))):4 states - 0ms
    ((conv_pell_pellb2(i,ii))&conv_pell_pellb2(j,jj)))&conv_pell_pellb2(n,nn))):8 stat

In [20]:
eval comp_pell2 n "?msd_pell Aj $abeq_pell2(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_pell2(i,j,n))=>i<=j):75 states - 4ms
  (A j (abeq_pell2(i,j,n))=>i<=j)):10 states - 9ms
Total computation time: 14ms.


### Apply the semigroup trick

In [21]:
%SGT comp_pell2 msd_pell Comp_pell2

Reading Result/comp_pell2.mpl
Rank: 4
6 states
values: [0, 1, 2, 3, 4]
Writing Word Automata Library/Comp_pell2.txt
Done.

## 3-abelian complexity

Define the 3-block map of pell and construct the conversion predicate
between both numeration systems.

In [22]:
%%python
s3 = block(s, 3)
ns3 = address(s3, "pellb3")
ns3.gen_ns()
(ns-ns3).gen_dfa("conv_pell_pellb3")

>>> Address automaton for block(subst(0->001, 1->0), 3)
  DT polynomial: X^2-2X-1 (n=2)
  θ=2.414213562373095
  DT automaton: 4 states, 4 final states, 10 transitions.

>>> address(block(subst(0->001, 1->0), 3), 'pellb3') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 4 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.N9a7399yLZ/Walnut/Custom Bases/msd_pellb3.txt in format Walnut
  +address(block(subst(0->001, 1->0), 3), 'pellb3')+address(block(subst(0->001, 1->0), 3), 'pellb3')-address(block(subst(0->001, 1->0), 3), 'pellb3') is generating it's dfa under name msd_pellb3_addition
  Combination polynomial is X^2-2X-1
  θ=2.414213562373095
  Combination automaton: 64 states, 64 final states, 1000 transitions.
  (P,u) vector bound: (0.0, 0.0) + (1.8106601717798219, 3.5177669529663698)*C
  Fast poly bound: C in [(-2, -2), (4, 4)] => (14.485281374238575, 28.142135623730958)
Expanding DFA...done.
  Flattened automaton : 1080 states, 

Translate the border condition predicate into the current numeration
system.

In [23]:
def bordercond3 "?msd_pellb3 (?msd_pell Eii,jj,kk,nn 
($conv_pell_pellb3(?msd_pell ii, ?msd_pellb3 i) & 
$conv_pell_pellb3(?msd_pell jj, ?msd_pellb3 j) & 
$conv_pell_pellb3(?msd_pell kk, ?msd_pellb3 k) & 
$conv_pell_pellb3(?msd_pell nn, ?msd_pellb3 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_pell_pellb3(ii,i))&conv_pell_pellb3(jj,j))):4 states - 1ms
 ((conv_pell_pellb3(ii,i))&conv_pell_pellb3(jj,j)))&conv_pell_pellb3(kk,k))):8 states - 0ms
  (((conv_pell_pellb3(ii,i))&conv_pell_pellb3(jj,j)))&conv_pell_pellb3(kk,k)))&conv_pell_pellb3(nn,n))):16 states - 4ms
   ((((conv_pell_pellb3(ii,i))&conv_pell_pellb3(jj,j)))&conv_pell_pellb3(kk,k)))&conv_pell_pellb3(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 88ms
    (E ii , jj , kk , nn ((((conv_pell_pellb3(ii,i))&conv_pell_pellb3(jj,j)))&conv_pell_pellb3(kk,k)))&conv_pell_pellb3(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 2ms
Total computation time: 96ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Pellb3.

In [24]:
%%python
for (i,a) in enumerate(ns3.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s3, ns3.ns, **w)
    (parikh - ns3).gen_dfa(f"pellb3p{i}")

>>> Address automaton for block(subst(0->001, 1->0), 3)
  DT polynomial: X^3-2X^2-X (n=3)
  θ=2.414213562373095 (X^1)
  DT automaton: 4 states, 4 final states, 10 transitions.

>>> +address(block(subst(0->001, 1->0), 3), 'pellb3', '_': 0, 'a': 1)-address(block(subst(0->001, 1->0), 3), 'pellb3') is generating it's dfa under name pellb3p0
  Combination polynomial is X^3-2X^2-X
  θ=2.414213562373095 (X^1)
  Combination automaton: 16 states, 16 final states, 100 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.164213562373096, 8.992640687119287, 18.235281374238575)*C
  Fast poly bound: C in [(-1, -3, -2), (0, 0, 1)] => (30.985281374238575, 53.955844122715725, 109.41168824543145)
Expanding DFA...done.
  Flattened automaton : 802 states, 20 final states, 1943 transitions.
  Trimmed flattened automaton : 35 states, 20 final states, 85 transitions.
  Real bound is [1, 3, 7] (vs [30.985281374238575, 53.955844122715725, 109.41168824543145]).
  After minimization: 13 states, 5 final states

### Compute the complexity function

Use Walnut to generate a linear representation for the 3-abelian
complexity.

In [25]:
def fac0 "?msd_pellb3 Ex,y $pellb3p0(i,x) & $pellb3p0(i+n,y) & z+x=y":
def fac1 "?msd_pellb3 Ex,y $pellb3p1(i,x) & $pellb3p1(i+n,y) & z+x=y":
def fac2 "?msd_pellb3 Ex,y $pellb3p2(i,x) & $pellb3p2(i+n,y) & z+x=y":
def fac3 "?msd_pellb3 Ex,y $pellb3p3(i,x) & $pellb3p3(i+n,y) & z+x=y":

(pellb3p0(i,x))&pellb3p0((i+n),y))):223 states - 3ms
 (z+x)=y:16 states - 0ms
  ((pellb3p0(i,x))&pellb3p0((i+n),y)))&(z+x)=y):378 states - 9ms
   (E x , y ((pellb3p0(i,x))&pellb3p0((i+n),y)))&(z+x)=y)):84 states - 1ms
Total computation time: 14ms.

(pellb3p1(i,x))&pellb3p1((i+n),y))):191 states - 2ms
 (z+x)=y:16 states - 0ms
  ((pellb3p1(i,x))&pellb3p1((i+n),y)))&(z+x)=y):331 states - 8ms
   (E x , y ((pellb3p1(i,x))&pellb3p1((i+n),y)))&(z+x)=y)):52 states - 1ms
Total computation time: 12ms.

(pellb3p2(i,x))&pellb3p2((i+n),y))):143 states - 1ms
 (z+x)=y:16 states - 0ms
  ((pellb3p2(i,x))&pellb3p2((i+n),y)))&(z+x)=y):253 states - 5ms
   (E x , y ((pellb3p2(i,x))&pellb3p2((i+n),y)))&(z+x)=y)):44 states - 1ms
Total computation time: 8ms.

(pellb3p3(i,x))&pellb3p3((i+n),y))):404 states - 3ms
 (z+x)=y:16 states - 0ms
  ((pellb3p3(i,x))&pellb3p3((i+n),y)))&(z+x)=y):897 states - 21ms
   (E x , y ((pellb3p3(i,x))&pellb3p3((i+n),y)))&(z+x)=y)):133 states - 4ms
Total computation time: 29ms.


In [26]:
def min0 "?msd_pellb3 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_pellb3 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_pellb3 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_pellb3 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):267 states - 47ms
  (A j , y (fac0(j,n,y))=>y>=x)):14 states - 39ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):60 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 88ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):191 states - 31ms
  (A j , y (fac1(j,n,y))=>y>=x)):14 states - 27ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):41 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 59ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):174 states - 27ms
  (A j , y (fac2(j,n,y))=>y>=x)):14 states - 23ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):34 states - 1ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 52ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):444 states - 58ms
  (A j , y (fac3(j,n,y))=>y>=x)):22 states - 55ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):87 stat

In [27]:
def diff0 "?msd_pellb3 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_pellb3 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_pellb3 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_pellb3 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):102 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):93 states - 2ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):25 states - 0ms
Total computation time: 3ms.

(min1(n,x))&fac1(i,n,y))):69 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):66 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):16 states - 0ms
Total computation time: 2ms.

(min2(n,x))&fac2(i,n,y))):58 states - 0ms
 (z+x)=y:16 states - 1ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):55 states - 1ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):14 states - 0ms
Total computation time: 2ms.

(min3(n,x))&fac3(i,n,y))):159 states - 2ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):148 states - 3ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):26 states - 1ms
Total computation time: 7ms.


In [28]:
def abeq_pellb3 "?msd_pellb3 $bordercond3(i,j,3,n+2) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z))":
def abeq_pell3 "?msd_pell (n<2 & $feq_pell(i,j,n)) | (n>=2 & (?msd_pellb3 Eii,jj,nn ($conv_pell_pellb3(?msd_pell i, ?msd_pellb3 ii) & $conv_pell_pellb3(?msd_pell j, ?msd_pellb3 jj) & $conv_pell_pellb3(?msd_pell n, ?msd_pellb3 nn) & $abeq_pellb3(ii,jj,nn-2))))":

computed ~:1 states - 0ms
computed ~:2 states - 1ms
(diff0(i,n,z))&diff0(j,n,z))):111 states - 2ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):104 states - 1ms
  (bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):75 states - 1ms
   (diff1(i,n,z))&diff1(j,n,z))):60 states - 0ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):51 states - 1ms
     ((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):69 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):46 states - 1ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):37 states - 1ms
        (((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):63 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):110 states - 2ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):100 states - 1ms
           ((((bordercond3(i,j,3,(n+2)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,

In [29]:
eval comp_pell3 n "?msd_pell Aj $abeq_pell3(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_pell3(i,j,n))=>i<=j):82 states - 4ms
  (A j (abeq_pell3(i,j,n))=>i<=j)):13 states - 4ms
Total computation time: 8ms.


### Apply the semigroup trick

In [30]:
%SGT comp_pell3 msd_pell Comp_pell3

Reading Result/comp_pell3.mpl
Rank: 5
8 states
values: [0, 1, 2, 3, 4, 5, 6]
Writing Word Automata Library/Comp_pell3.txt
Done.

## 4-abelian complexity

Define the 4-block map of pell and construct the conversion predicate
between both numeration systems.

In [31]:
%%python
s4 = block(s, 4)
ns4 = address(s4, "pellb4")
ns4.gen_ns()
(ns-ns4).gen_dfa("conv_pell_pellb4")

>>> Address automaton for block(subst(0->001, 1->0), 4)
  DT polynomial: X^2-2X-1 (n=2)
  θ=2.414213562373095
  DT automaton: 5 states, 5 final states, 11 transitions.

>>> address(block(subst(0->001, 1->0), 4), 'pellb4') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 4 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.N9a7399yLZ/Walnut/Custom Bases/msd_pellb4.txt in format Walnut
  +address(block(subst(0->001, 1->0), 4), 'pellb4')+address(block(subst(0->001, 1->0), 4), 'pellb4')-address(block(subst(0->001, 1->0), 4), 'pellb4') is generating it's dfa under name msd_pellb4_addition
  Combination polynomial is X^2-2X-1
  θ=2.414213562373095
  Combination automaton: 125 states, 125 final states, 1331 transitions.
  (P,u) vector bound: (0.0, 0.0) + (1.8106601717798219, 3.5177669529663698)*C
  Fast poly bound: C in [(-2, -2), (4, 4)] => (14.485281374238575, 28.142135623730958)
Expanding DFA...done.
  Flattened automaton : 1660 states

Translate the border condition predicate into the current numeration
system.

In [32]:
def bordercond4 "?msd_pellb4 (?msd_pell Eii,jj,kk,nn 
($conv_pell_pellb4(?msd_pell ii, ?msd_pellb4 i) & 
$conv_pell_pellb4(?msd_pell jj, ?msd_pellb4 j) & 
$conv_pell_pellb4(?msd_pell kk, ?msd_pellb4 k) & 
$conv_pell_pellb4(?msd_pell nn, ?msd_pellb4 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_pell_pellb4(ii,i))&conv_pell_pellb4(jj,j))):4 states - 0ms
 ((conv_pell_pellb4(ii,i))&conv_pell_pellb4(jj,j)))&conv_pell_pellb4(kk,k))):8 states - 0ms
  (((conv_pell_pellb4(ii,i))&conv_pell_pellb4(jj,j)))&conv_pell_pellb4(kk,k)))&conv_pell_pellb4(nn,n))):16 states - 5ms
   ((((conv_pell_pellb4(ii,i))&conv_pell_pellb4(jj,j)))&conv_pell_pellb4(kk,k)))&conv_pell_pellb4(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 104ms
    (E ii , jj , kk , nn ((((conv_pell_pellb4(ii,i))&conv_pell_pellb4(jj,j)))&conv_pell_pellb4(kk,k)))&conv_pell_pellb4(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 2ms
Total computation time: 112ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Pellb4.

In [33]:
%%python
for (i,a) in enumerate(ns4.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s4, ns4.ns, **w)
    (parikh - ns4).gen_dfa(f"pellb4p{i}")

>>> Address automaton for block(subst(0->001, 1->0), 4)
  DT polynomial: X^3-2X^2-X (n=3)
  θ=2.414213562373095 (X^1)
  DT automaton: 5 states, 5 final states, 11 transitions.

>>> +address(block(subst(0->001, 1->0), 4), 'pellb4', '_': 0, 'a': 1)-address(block(subst(0->001, 1->0), 4), 'pellb4') is generating it's dfa under name pellb4p0
  Combination polynomial is X^3-2X^2-X
  θ=2.414213562373095 (X^1)
  Combination automaton: 25 states, 25 final states, 121 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.164213562373096, 8.992640687119287, 18.235281374238575)*C
  Fast poly bound: C in [(-1, -3, -2), (0, 0, 1)] => (30.985281374238575, 53.955844122715725, 109.41168824543145)
Expanding DFA...done.
  Flattened automaton : 998 states, 25 final states, 2126 transitions.
  Trimmed flattened automaton : 43 states, 25 final states, 93 transitions.
  Real bound is [1, 3, 7] (vs [30.985281374238575, 53.955844122715725, 109.41168824543145]).
  After minimization: 13 states, 5 final states

### Compute the complexity function

Use Walnut to generate a linear representation for the 4-abelian
complexity.

In [34]:
def fac0 "?msd_pellb4 Ex,y $pellb4p0(i,x) & $pellb4p0(i+n,y) & z+x=y":
def fac1 "?msd_pellb4 Ex,y $pellb4p1(i,x) & $pellb4p1(i+n,y) & z+x=y":
def fac2 "?msd_pellb4 Ex,y $pellb4p2(i,x) & $pellb4p2(i+n,y) & z+x=y":
def fac3 "?msd_pellb4 Ex,y $pellb4p3(i,x) & $pellb4p3(i+n,y) & z+x=y":
def fac4 "?msd_pellb4 Ex,y $pellb4p4(i,x) & $pellb4p4(i+n,y) & z+x=y":

(pellb4p0(i,x))&pellb4p0((i+n),y))):223 states - 1ms
 (z+x)=y:16 states - 1ms
  ((pellb4p0(i,x))&pellb4p0((i+n),y)))&(z+x)=y):378 states - 7ms
   (E x , y ((pellb4p0(i,x))&pellb4p0((i+n),y)))&(z+x)=y)):84 states - 1ms
Total computation time: 12ms.

(pellb4p1(i,x))&pellb4p1((i+n),y))):191 states - 1ms
 (z+x)=y:16 states - 0ms
  ((pellb4p1(i,x))&pellb4p1((i+n),y)))&(z+x)=y):331 states - 7ms
   (E x , y ((pellb4p1(i,x))&pellb4p1((i+n),y)))&(z+x)=y)):52 states - 1ms
Total computation time: 12ms.

(pellb4p2(i,x))&pellb4p2((i+n),y))):188 states - 2ms
 (z+x)=y:16 states - 0ms
  ((pellb4p2(i,x))&pellb4p2((i+n),y)))&(z+x)=y):419 states - 11ms
   (E x , y ((pellb4p2(i,x))&pellb4p2((i+n),y)))&(z+x)=y)):99 states - 2ms
Total computation time: 16ms.

(pellb4p3(i,x))&pellb4p3((i+n),y))):404 states - 3ms
 (z+x)=y:16 states - 1ms
  ((pellb4p3(i,x))&pellb4p3((i+n),y)))&(z+x)=y):897 states - 20ms
   (E x , y ((pellb4p3(i,x))&pellb4p3((i+n),y)))&(z+x)=y)):133 states - 4ms
Total computation time: 29ms.

(

In [35]:
def min0 "?msd_pellb4 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_pellb4 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_pellb4 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_pellb4 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_pellb4 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):267 states - 44ms
  (A j , y (fac0(j,n,y))=>y>=x)):14 states - 30ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):60 states - 1ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 75ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):191 states - 32ms
  (A j , y (fac1(j,n,y))=>y>=x)):14 states - 23ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):41 states - 0ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 56ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):323 states - 45ms
  (A j , y (fac2(j,n,y))=>y>=x)):17 states - 38ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):63 states - 0ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 85ms.

y>=x:6 states - 0ms
 (fac3(j,n,y))=>y>=x):444 states - 70ms
  (A j , y (fac3(j,n,y))=>y>=x)):22 states - 51ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):87 stat

In [36]:
def diff0 "?msd_pellb4 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_pellb4 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_pellb4 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_pellb4 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_pellb4 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):102 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):93 states - 2ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):25 states - 1ms
Total computation time: 4ms.

(min1(n,x))&fac1(i,n,y))):69 states - 1ms
 (z+x)=y:16 states - 1ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):66 states - 1ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):16 states - 0ms
Total computation time: 3ms.

(min2(n,x))&fac2(i,n,y))):125 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):114 states - 3ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):37 states - 0ms
Total computation time: 4ms.

(min3(n,x))&fac3(i,n,y))):159 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):148 states - 3ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):26 states - 1ms
Total computation time: 6ms.

(min4(n,x))&fac4(i,n,y))):164 states - 2ms
 (z+x)=y:16 states - 0ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)=y):150 states - 3ms
   (E x 

In [37]:
def abeq_pellb4 "?msd_pellb4 $bordercond4(i,j,4,n+3) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z))":
def abeq_pell4 "?msd_pell (n<3 & $feq_pell(i,j,n)) | (n>=3 & (?msd_pellb4 Eii,jj,nn ($conv_pell_pellb4(?msd_pell i, ?msd_pellb4 ii) & $conv_pell_pellb4(?msd_pell j, ?msd_pellb4 jj) & $conv_pell_pellb4(?msd_pell n, ?msd_pellb4 nn) & $abeq_pellb4(ii,jj,nn-3))))":

computed ~:1 states - 1ms
computed ~:2 states - 0ms
(diff0(i,n,z))&diff0(j,n,z))):111 states - 1ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):104 states - 2ms
  (bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):75 states - 1ms
   (diff1(i,n,z))&diff1(j,n,z))):60 states - 0ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):51 states - 1ms
     ((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):71 states - 1ms
      (diff2(i,n,z))&diff2(j,n,z))):145 states - 3ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):115 states - 2ms
        (((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):66 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):110 states - 2ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):100 states - 1ms
           ((((bordercond4(i,j,4,(n+3)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(

In [38]:
eval comp_pell4 n "?msd_pell Aj $abeq_pell4(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_pell4(i,j,n))=>i<=j):85 states - 5ms
  (A j (abeq_pell4(i,j,n))=>i<=j)):15 states - 4ms
Total computation time: 10ms.


### Apply the semigroup trick

In [39]:
%SGT comp_pell4 msd_pell Comp_pell4

Reading Result/comp_pell4.mpl
Rank: 6
10 states
values: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Writing Word Automata Library/Comp_pell4.txt
Done.

## 5-abelian complexity

Define the 5-block map of pell and construct the conversion predicate
between both numeration systems.

In [40]:
%%python
s5 = block(s, 5)
ns5 = address(s5, "pellb5")
ns5.gen_ns()
(ns-ns5).gen_dfa("conv_pell_pellb5")

>>> Address automaton for block(subst(0->001, 1->0), 5)
  DT polynomial: X^2-2X-1 (n=2)
  θ=2.414213562373095
  DT automaton: 6 states, 6 final states, 14 transitions.

>>> address(block(subst(0->001, 1->0), 5), 'pellb5') is generating it's NS
  Numeration system automaton: 2 states, 2 final states, 4 transitions.
  Writing /var/folders/l8/fphf93t95171xj2_rf0knxtc0000gn/T/tmp.N9a7399yLZ/Walnut/Custom Bases/msd_pellb5.txt in format Walnut
  +address(block(subst(0->001, 1->0), 5), 'pellb5')+address(block(subst(0->001, 1->0), 5), 'pellb5')-address(block(subst(0->001, 1->0), 5), 'pellb5') is generating it's dfa under name msd_pellb5_addition
  Combination polynomial is X^2-2X-1
  θ=2.414213562373095
  Combination automaton: 216 states, 216 final states, 2744 transitions.
  (P,u) vector bound: (0.0, 0.0) + (1.8106601717798219, 3.5177669529663698)*C
  Fast poly bound: C in [(-2, -2), (4, 4)] => (14.485281374238575, 28.142135623730958)
Expanding DFA...done.
  Flattened automaton : 2359 states

Translate the border condition predicate into the current numeration
system.

In [41]:
def bordercond5 "?msd_pellb5 (?msd_pell Eii,jj,kk,nn 
($conv_pell_pellb5(?msd_pell ii, ?msd_pellb5 i) & 
$conv_pell_pellb5(?msd_pell jj, ?msd_pellb5 j) & 
$conv_pell_pellb5(?msd_pell kk, ?msd_pellb5 k) & 
$conv_pell_pellb5(?msd_pell nn, ?msd_pellb5 n) & 
$bordercond(ii,jj,kk,nn)))":






(conv_pell_pellb5(ii,i))&conv_pell_pellb5(jj,j))):4 states - 0ms
 ((conv_pell_pellb5(ii,i))&conv_pell_pellb5(jj,j)))&conv_pell_pellb5(kk,k))):8 states - 0ms
  (((conv_pell_pellb5(ii,i))&conv_pell_pellb5(jj,j)))&conv_pell_pellb5(kk,k)))&conv_pell_pellb5(nn,n))):16 states - 5ms
   ((((conv_pell_pellb5(ii,i))&conv_pell_pellb5(jj,j)))&conv_pell_pellb5(kk,k)))&conv_pell_pellb5(nn,n)))&bordercond(ii,jj,kk,nn))):44 states - 105ms
    (E ii , jj , kk , nn ((((conv_pell_pellb5(ii,i))&conv_pell_pellb5(jj,j)))&conv_pell_pellb5(kk,k)))&conv_pell_pellb5(nn,n)))&bordercond(ii,jj,kk,nn)))):44 states - 2ms
Total computation time: 113ms.


### Compute Parikh vectors

First we compute the Parikh vectors for the prefixes of Pellb5.

In [42]:
%%python
for (i,a) in enumerate(ns5.alpha):
    w = {'_': 0}
    w[a] = 1
    parikh = address(s5, ns5.ns, **w)
    (parikh - ns5).gen_dfa(f"pellb5p{i}")

>>> Address automaton for block(subst(0->001, 1->0), 5)
  DT polynomial: X^3-2X^2-X (n=3)
  θ=2.414213562373095 (X^1)
  DT automaton: 6 states, 6 final states, 14 transitions.

>>> +address(block(subst(0->001, 1->0), 5), 'pellb5', '_': 0, 'a': 1)-address(block(subst(0->001, 1->0), 5), 'pellb5') is generating it's dfa under name pellb5p0
  Combination polynomial is X^3-2X^2-X
  θ=2.414213562373095 (X^1)
  Combination automaton: 36 states, 36 final states, 196 transitions.
  (P,u) vector bound: (0.0, 0.0, 0.0) + (5.164213562373096, 8.992640687119287, 18.235281374238575)*C
  Fast poly bound: C in [(-1, -3, -2), (0, 0, 1)] => (30.985281374238575, 53.955844122715725, 109.41168824543145)
Expanding DFA...done.
  Flattened automaton : 1191 states, 29 final states, 2663 transitions.
  Trimmed flattened automaton : 52 states, 29 final states, 117 transitions.
  Real bound is [1, 3, 7] (vs [30.985281374238575, 53.955844122715725, 109.41168824543145]).
  After minimization: 13 states, 5 final stat

### Compute the complexity function

Use Walnut to generate a linear representation for the 5-abelian
complexity.

In [43]:
def fac0 "?msd_pellb5 Ex,y $pellb5p0(i,x) & $pellb5p0(i+n,y) & z+x=y":
def fac1 "?msd_pellb5 Ex,y $pellb5p1(i,x) & $pellb5p1(i+n,y) & z+x=y":
def fac2 "?msd_pellb5 Ex,y $pellb5p2(i,x) & $pellb5p2(i+n,y) & z+x=y":
def fac3 "?msd_pellb5 Ex,y $pellb5p3(i,x) & $pellb5p3(i+n,y) & z+x=y":
def fac4 "?msd_pellb5 Ex,y $pellb5p4(i,x) & $pellb5p4(i+n,y) & z+x=y":
def fac5 "?msd_pellb5 Ex,y $pellb5p5(i,x) & $pellb5p5(i+n,y) & z+x=y":

(pellb5p0(i,x))&pellb5p0((i+n),y))):223 states - 2ms
 (z+x)=y:16 states - 1ms
  ((pellb5p0(i,x))&pellb5p0((i+n),y)))&(z+x)=y):378 states - 8ms
   (E x , y ((pellb5p0(i,x))&pellb5p0((i+n),y)))&(z+x)=y)):84 states - 1ms
Total computation time: 13ms.

(pellb5p1(i,x))&pellb5p1((i+n),y))):222 states - 3ms
 (z+x)=y:16 states - 0ms
  ((pellb5p1(i,x))&pellb5p1((i+n),y)))&(z+x)=y):486 states - 14ms
   (E x , y ((pellb5p1(i,x))&pellb5p1((i+n),y)))&(z+x)=y)):126 states - 2ms
Total computation time: 20ms.

(pellb5p2(i,x))&pellb5p2((i+n),y))):188 states - 2ms
 (z+x)=y:16 states - 1ms
  ((pellb5p2(i,x))&pellb5p2((i+n),y)))&(z+x)=y):419 states - 11ms
   (E x , y ((pellb5p2(i,x))&pellb5p2((i+n),y)))&(z+x)=y)):99 states - 2ms
Total computation time: 18ms.

(pellb5p3(i,x))&pellb5p3((i+n),y))):404 states - 4ms
 (z+x)=y:16 states - 0ms
  ((pellb5p3(i,x))&pellb5p3((i+n),y)))&(z+x)=y):897 states - 24ms
   (E x , y ((pellb5p3(i,x))&pellb5p3((i+n),y)))&(z+x)=y)):133 states - 3ms
Total computation time: 34ms.


In [44]:
def min0 "?msd_pellb5 Ei $fac0(i,n,x) & Aj,y $fac0(j,n,y) => y>=x":
def min1 "?msd_pellb5 Ei $fac1(i,n,x) & Aj,y $fac1(j,n,y) => y>=x":
def min2 "?msd_pellb5 Ei $fac2(i,n,x) & Aj,y $fac2(j,n,y) => y>=x":
def min3 "?msd_pellb5 Ei $fac3(i,n,x) & Aj,y $fac3(j,n,y) => y>=x":
def min4 "?msd_pellb5 Ei $fac4(i,n,x) & Aj,y $fac4(j,n,y) => y>=x":
def min5 "?msd_pellb5 Ei $fac5(i,n,x) & Aj,y $fac5(j,n,y) => y>=x":

y>=x:6 states - 0ms
 (fac0(j,n,y))=>y>=x):267 states - 43ms
  (A j , y (fac0(j,n,y))=>y>=x)):14 states - 33ms
   (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x))):60 states - 0ms
    (E i (fac0(i,n,x))&(A j , y (fac0(j,n,y))=>y>=x)))):12 states - 0ms
Total computation time: 77ms.

y>=x:6 states - 0ms
 (fac1(j,n,y))=>y>=x):405 states - 60ms
  (A j , y (fac1(j,n,y))=>y>=x)):17 states - 47ms
   (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x))):78 states - 1ms
    (E i (fac1(i,n,x))&(A j , y (fac1(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 110ms.

y>=x:6 states - 0ms
 (fac2(j,n,y))=>y>=x):323 states - 48ms
  (A j , y (fac2(j,n,y))=>y>=x)):17 states - 37ms
   (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x))):63 states - 1ms
    (E i (fac2(i,n,x))&(A j , y (fac2(j,n,y))=>y>=x)))):17 states - 0ms
Total computation time: 87ms.

y>=x:6 states - 1ms
 (fac3(j,n,y))=>y>=x):444 states - 67ms
  (A j , y (fac3(j,n,y))=>y>=x)):22 states - 50ms
   (fac3(i,n,x))&(A j , y (fac3(j,n,y))=>y>=x))):87 sta

In [45]:
def diff0 "?msd_pellb5 Ex,y $min0(n,x) & $fac0(i,n,y) & z+x=y":
def diff1 "?msd_pellb5 Ex,y $min1(n,x) & $fac1(i,n,y) & z+x=y":
def diff2 "?msd_pellb5 Ex,y $min2(n,x) & $fac2(i,n,y) & z+x=y":
def diff3 "?msd_pellb5 Ex,y $min3(n,x) & $fac3(i,n,y) & z+x=y":
def diff4 "?msd_pellb5 Ex,y $min4(n,x) & $fac4(i,n,y) & z+x=y":
def diff5 "?msd_pellb5 Ex,y $min5(n,x) & $fac5(i,n,y) & z+x=y":

(min0(n,x))&fac0(i,n,y))):102 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min0(n,x))&fac0(i,n,y)))&(z+x)=y):93 states - 2ms
   (E x , y ((min0(n,x))&fac0(i,n,y)))&(z+x)=y)):25 states - 1ms
Total computation time: 4ms.

(min1(n,x))&fac1(i,n,y))):158 states - 2ms
 (z+x)=y:16 states - 0ms
  ((min1(n,x))&fac1(i,n,y)))&(z+x)=y):146 states - 4ms
   (E x , y ((min1(n,x))&fac1(i,n,y)))&(z+x)=y)):46 states - 1ms
Total computation time: 8ms.

(min2(n,x))&fac2(i,n,y))):125 states - 2ms
 (z+x)=y:16 states - 0ms
  ((min2(n,x))&fac2(i,n,y)))&(z+x)=y):114 states - 4ms
   (E x , y ((min2(n,x))&fac2(i,n,y)))&(z+x)=y)):37 states - 1ms
Total computation time: 8ms.

(min3(n,x))&fac3(i,n,y))):159 states - 1ms
 (z+x)=y:16 states - 0ms
  ((min3(n,x))&fac3(i,n,y)))&(z+x)=y):148 states - 2ms
   (E x , y ((min3(n,x))&fac3(i,n,y)))&(z+x)=y)):26 states - 1ms
Total computation time: 5ms.

(min4(n,x))&fac4(i,n,y))):216 states - 2ms
 (z+x)=y:16 states - 0ms
  ((min4(n,x))&fac4(i,n,y)))&(z+x)=y):198 states - 4ms
   (E 

In [46]:
def abeq_pellb5 "?msd_pellb5 $bordercond5(i,j,5,n+4) & (Ez $diff0(i,n,z) &  $diff0(j,n,z)) & (Ez $diff1(i,n,z) &  $diff1(j,n,z)) & (Ez $diff2(i,n,z) &  $diff2(j,n,z)) & (Ez $diff3(i,n,z) &  $diff3(j,n,z)) & (Ez $diff4(i,n,z) &  $diff4(j,n,z)) & (Ez $diff5(i,n,z) &  $diff5(j,n,z))":
def abeq_pell5 "?msd_pell (n<4 & $feq_pell(i,j,n)) | (n>=4 & (?msd_pellb5 Eii,jj,nn ($conv_pell_pellb5(?msd_pell i, ?msd_pellb5 ii) & $conv_pell_pellb5(?msd_pell j, ?msd_pellb5 jj) & $conv_pell_pellb5(?msd_pell n, ?msd_pellb5 nn) & $abeq_pellb5(ii,jj,nn-4))))":

computed ~:1 states - 0ms
computed ~:2 states - 1ms
(diff0(i,n,z))&diff0(j,n,z))):111 states - 2ms
 (E z (diff0(i,n,z))&diff0(j,n,z)))):104 states - 2ms
  (bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z))))):90 states - 1ms
   (diff1(i,n,z))&diff1(j,n,z))):234 states - 4ms
    (E z (diff1(i,n,z))&diff1(j,n,z)))):224 states - 4ms
     ((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z))))):89 states - 2ms
      (diff2(i,n,z))&diff2(j,n,z))):145 states - 3ms
       (E z (diff2(i,n,z))&diff2(j,n,z)))):115 states - 2ms
        (((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff2(j,n,z))))):81 states - 1ms
         (diff3(i,n,z))&diff3(j,n,z))):110 states - 2ms
          (E z (diff3(i,n,z))&diff3(j,n,z)))):100 states - 1ms
           ((((bordercond5(i,j,5,(n+4)))&(E z (diff0(i,n,z))&diff0(j,n,z)))))&(E z (diff1(i,n,z))&diff1(j,n,z)))))&(E z (diff2(i,n,z))&diff

In [47]:
eval comp_pell5 n "?msd_pell Aj $abeq_pell5(i,j,n) => i<=j":

i<=j:6 states - 0ms
 (abeq_pell5(i,j,n))=>i<=j):105 states - 5ms
  (A j (abeq_pell5(i,j,n))=>i<=j)):22 states - 4ms
Total computation time: 10ms.


### Apply the semigroup trick

In [48]:
%SGT comp_pell5 msd_pell Comp_pell5

Reading Result/comp_pell5.mpl
Rank: 7
12 states
values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Writing Word Automata Library/Comp_pell5.txt
Done.

## Check it out!

In [49]:
%%python
from itertools import product
from ratser.walimp import from_walnut

def valid(a, h, u):
    cur=a
    for c in u:
        v = h[cur]
        if c >= len(v):
            return False
        cur = v[c]
    return True

def enumrepr(a, h, k):
    alpha=list(range(max(map(len,h.values()))))
    for u in product(alpha, repeat=k):
        if valid(a, h, u):
            yield ''.join(map(str,u))

l = []
lbl = []
for k in range(1,6):
    lbl.append(str(k))
    with open(P(os.environ["WALNUT_HOME"]) / P(f"Result/comp_pell{k}.mpl")) as f:
            ser = from_walnut(f)
            l.append(ser)

(a,h)=s.subst()
print(f"{'n':>4}", *[f"{x+'-ab(pell)':>15}" for x in lbl])
print('-'*(4+16*len(lbl)))
for (i, u) in enumerate(enumrepr(a, h, 8)):
    print(f"{i:4}", *[ f"{int(s.value(u)):15}" for s in l ])

   n      1-ab(pell)      2-ab(pell)      3-ab(pell)      4-ab(pell)      5-ab(pell)
------------------------------------------------------------------------------------
   0               1               1               1               1               1
   1               2               2               2               2               2
   2               2               3               3               3               3
   3               2               4               4               4               4
   4               2               4               5               5               5
   5               2               4               6               6               6
   6               2               4               6               7               7
   7               2               4               6               8               8
   8               2               4               6               8               9
   9               2               4               6             

## Let’s wrap it up!

In [50]:
%%shell
cd $WALNUT_HOME
cat << EOF | tr '\n' '\0' | tar cvzf /tmp/abelian.tar.gz --null -T -
Custom Bases/msd_pell.txt
Custom Bases/msd_pell_addition.txt
Word Automata Library/Pell.txt
Automata Library/feq_pell.txt
Result/comp_pell.mpl
Word Automata Library/Comp_pell1.txt
Result/comp_pell1.mpl
Automata Library/abeq_pell1.txt
Word Automata Library/Comp_pell2.txt
Result/comp_pell2.mpl
Automata Library/abeq_pell2.txt
Word Automata Library/Comp_pell3.txt
Result/comp_pell3.mpl
Automata Library/abeq_pell3.txt
Word Automata Library/Comp_pell4.txt
Result/comp_pell4.mpl
Automata Library/abeq_pell4.txt
Word Automata Library/Comp_pell5.txt
Result/comp_pell5.mpl
Automata Library/abeq_pell5.txt
EOF

a Custom Bases/msd_pell.txt
a Custom Bases/msd_pell_addition.txt
a Word Automata Library/Pell.txt
a Automata Library/feq_pell.txt
a Result/comp_pell.mpl
a Word Automata Library/Comp_pell1.txt
a Result/comp_pell1.mpl
a Automata Library/abeq_pell1.txt
a Word Automata Library/Comp_pell2.txt
a Result/comp_pell2.mpl
a Automata Library/abeq_pell2.txt
a Word Automata Library/Comp_pell3.txt
a Result/comp_pell3.mpl
a Automata Library/abeq_pell3.txt
a Word Automata Library/Comp_pell4.txt
a Result/comp_pell4.mpl
a Automata Library/abeq_pell4.txt
a Word Automata Library/Comp_pell5.txt
a Result/comp_pell5.mpl
a Automata Library/abeq_pell5.txt
